In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [38]:
df = pd.read_csv('train.csv')
df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [39]:
max_features = 1000000
Vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_features, output_mode='tf_idf', ngrams=2)
Vectorizer.adapt(np.array(df['text']))
vocab = Vectorizer.get_vocabulary()

In [29]:
model = tf.keras.Sequential()
model.add(Vectorizer)
    
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
    
model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(1e-3),
             metrics=['accuracy'])

In [22]:
X = df['text']
y = df['author']

In [23]:
training_labels_bools = []

for author in y:
    if "EAP" == author:
        training_labels_bools.append([1,0,0])
    elif "HPL" == author:
        training_labels_bools.append([0,1,0])
    elif "MWS" == author:
        training_labels_bools.append([0,0,1])

In [24]:
y = training_labels_bools
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

In [30]:
model.fit(list(X_train), y_train, epochs=1, batch_size=64,
         validation_data=(list(X_test),y_test),
         validation_steps=10)

230/230 [==============================] - 8s 34ms/step - loss: 0.5615 - accuracy: 0.7732 - val_loss: 0.3870 - val_accuracy: 0.8547


In [27]:
pred = model.predict(df['text'])
pred

array([[9.9972039e-01, 7.3953168e-05, 2.0560717e-04],
       [1.5727557e-01, 6.8825155e-01, 1.5447287e-01],
       [9.9446625e-01, 5.4519046e-03, 8.1904189e-05],
       ...,
       [9.7639024e-01, 1.4058221e-02, 9.5514208e-03],
       [5.0667745e-01, 3.6083096e-01, 1.3249153e-01],
       [1.0381318e-02, 9.7946578e-01, 1.0152815e-02]], dtype=float32)

In [15]:
ngram_ensemble = pd.DataFrame({
    "id": df["id"],
    "EAP_ngram": pred[:, 0],
    "HPL_ngram": pred[:, 1],
    "MWS_ngram": pred[:, 2],
    "actual_author": df['author']
})
ngram_ensemble.head()

,id,EAP_ngram,HPL_ngram,MWS_ngram,actual_author
0,id26305,0.998092,0.000659,0.001248,EAP
1,id17569,0.013822,0.908279,0.077899,HPL
2,id11008,0.999991,0.000007,0.000002,EAP
3,id27763,0.001144,0.001588,0.997268,MWS
4,id12958,0.000697,0.998542,0.000761,HPL


In [16]:
ngram_ensemble.to_csv("ngram_ensemble.csv", index=False)